In [1]:
import os # package for using operating system
from dotenv import load_dotenv
from snowflake.snowpark.session import Session # package for building and using Snowflake sessions
# from snowflake.cortex import Complete
import pandas as pd # package for data manipulation
import json # package to work with .json

In [16]:
# load .env file
load_dotenv(override=True)

# set up connection parameters for Snowflake connection
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'], # credentials
    "user": os.environ['SNOWFLAKE_USER'], # credentials
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'], # credentials
    "role": 'SYSADMIN', # specifiy Snowflake role
    "warehouse": 'COMPUTE_WH', # choose warehouse for computation
    'paramstyle': 'qmark' # set parameter style --> ?
}

snowflake_session = Session.builder.configs(connection_params).create() # build Snowflake session with connection parameters
print('Snowflake sessions is build.')

Snowflake sessions is build.


In [3]:
# test query
query = f"select try_to_date('20250608') as response"
respsone = snowflake_session.sql(query=query).collect()

In [8]:
respsone[0]['RESPONSE']

datetime.date(1970, 8, 23)

In [4]:
res = json.loads(respsone[0]['RESPONSE'])

TypeError: the JSON object must be str, bytes or bytearray, not date

In [15]:
model = 'llama3.1-8b'
# specify llm parameters for summary creation
model_params = {
   'temperature': 0, # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex --> Internetrecherche hat keine anderen Empfehlungen ergeben
   # 'top_p': # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex
    'max_tokens': 40
}

In [17]:
prompt = 'Write me a poem about curcumbers'

In [18]:
query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE( 
                ?,
                [
                    {{
                        'role': 'user', 
                        'content': ?
                    }}
                ],
                {{
                    'temperature': ?,
                    'max_tokens':  ?
                }} 
            ) AS response
        """
# query parameters
# model --> llm model
# prompt --> input prompt with instructions for llm
# model_params['temperature'] --> controll randomness in llm response
# model_params['max_tokens'] --> limit llm response length
# sql().collect() --> execute sql statement and return a DataFrame (no Pandas dataframe) as list of row objects https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.collect#snowflake.snowpark.DataFrame.collect
response = snowflake_session.sql(query, params=[model, prompt, model_params['temperature'], model_params['max_tokens']]).collect()
res = json.loads(response[0]['RESPONSE']) # load response as json object and save it in 'res'
# split 'res' parts and save them into multiple variabels
message = res['choices'][0]['messages'] 

In [19]:
res

{'choices': [{'messages': "In gardens green, where sunbeams play,\nA refreshing sight, in summer's day,\nThe cucumber, a treasure rare,\nA crunchy delight, beyond compare.\n\nIts skin, a gentle, em"}],
 'created': 1749975890,
 'model': 'llama3.1-8b',
 'usage': {'completion_tokens': 40, 'prompt_tokens': 19, 'total_tokens': 59}}

In [20]:
snowflake_session.close()